# Assignment Set 2

*Authors*: Myriam Belkhatir, Salomé Poulain, Shania Sinha

## Import Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from importlib import reload
from matplotlib.animation import FuncAnimation, PillowWriter


%matplotlib inline

In [ ]:
from src.deterministic_dla import DeterministicDLA

In [ ]:
# Run the deterministic DLA simulation
dla = DeterministicDLA(grid_size=100, eta=1.0, max_iterations=2000)
dla.run_simulation()

## Gray-Scott model

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from importlib import reload
from matplotlib.animation import FuncAnimation, PillowWriter

from src.gray_scott import GrayScottModel

In [ ]:
N = 200
steps = 15000
Du = 0.16
Dv = 0.08

### Explore broad range of f and k

In [ ]:
# Define parameter ranges from low to high
F_grid_extended = np.linspace(0.03, 0.14, 3) 
k_grid_extended = np.linspace(0.045, 0.065, 3)

# Create meshgrid - switch order to get F on y-axis, k on x-axis
k_mesh_ext, F_mesh_ext = np.meshgrid(k_grid_extended, F_grid_extended)
F_points_ext = F_mesh_ext.flatten()
k_points_ext = k_mesh_ext.flatten()

# Create descriptions
descriptions = [f"F={f:.3f}, k={k:.3f}" for f, k in zip(F_points_ext, k_points_ext)]

# Create models for each parameter set
models = []
for f, k, desc in zip(F_points_ext, k_points_ext, descriptions):
    model = GrayScottModel(N=N, Du=Du, Dv=Dv, f=f, k=k)
    models.append(model)

In [ ]:
# Run simulations
for i, model in enumerate(models):
    model.run(steps=steps, save_interval=50)

In [ ]:
# Create a 3x3 plot of the final U field
fig, axs = plt.subplots(3, 3, figsize=(10, 10), constrained_layout=True, sharex=True, sharey=True)

for i, model in enumerate(models):
    u_final, _ = model.simulation[-1] 
    row = 2 - (i // 3)  # Reverse row index for bottom-up ordering
    col = i % 3        
    ax = axs[row, col]  
    im = ax.imshow(u_final, cmap="viridis", vmin=0, vmax=1)
    ax.set_title(f"{descriptions[i]}", fontsize=18)  
    ax.axis("off")

# Add a shared colorbar with proper font size configuration
cbar = fig.colorbar(im, ax=axs, orientation='vertical', fraction=0.046, pad=0.04)
cbar.ax.tick_params(labelsize=18) 

plt.savefig("results/gray_scott/gray_scott_3x3_final_states.pgf")
plt.show()

### Explore zoomed in region with different f and k values for distinct patterns

In [ ]:
# Format: (Du, Dv, f, k, description)
# Parameters from Har-Shemesh et al. (2015)
parameter_sets = [
    (0.0416, 0.0625),  
    (0.0392, 0.0649),  
    (0.0175, 0.0504),  
    (0.0295, 0.0561)  
]

# Complete the parameter sets with Du, Dv, and descriptions
descriptions = ["Worm-like structures", "Maze-like patterns", "Transition zone", "Edge of pattern-forming region"]
complete_params = [(Du, Dv, f, k, desc) for (f, k), desc in zip(parameter_sets, descriptions)]

# Create the models for each set of parameters
models = []
for Du, Dv, f, k, desc in complete_params:
    model = GrayScottModel(N=N, Du=Du, Dv=Dv, f=f, k=k)
    models.append(model)

In [ ]:
# Run simulations
for i, model in enumerate(models):
    model.run(steps=steps, save_interval=50)

In [ ]:
# Create a 2x2 plot of the final U field from the stored simulation snapshots
fig, axs = plt.subplots(2, 2, figsize=(10, 10), constrained_layout=True)

for i, model in enumerate(models):
    # Get the final snapshot stored in the simulation attribute (u, v)
    u_final, _ = model.simulation[-1]
    ax = axs[i // 2, i % 2]
    im = ax.imshow(u_final, cmap="viridis", vmin=0, vmax=1)
    
    # Extract f and k values from parameter_sets for the title
    f_val = parameter_sets[i][0]
    k_val = parameter_sets[i][1]
    ax.set_title(f"F={f_val:.4f}, k={k_val:.4f}", fontsize=18)
    ax.axis("off")

# Add a single colorbar for the entire figure
cbar = fig.colorbar(im, ax=axs, orientation='vertical', fraction=0.046, pad=0.04)
cbar.ax.tick_params(labelsize=18)

plt.savefig("results/gray_scott/gray_scott_2x2_final_states.pgf")
plt.show()

In [ ]:
# Create and save GIF animations for each simulation
for i, model in enumerate(models):
    ani = model.create_animation(interval=50)
    
    # Use the complete_params list which contains descriptions
    description = parameter_sets[i]
    filename = f"results/gray_scott/gif_{description}.gif"
    
    print(f"Saving animation as {filename} ...")
    ani.save(filename, writer=PillowWriter(fps=20))

# References

Har-Shemesh et al., 2015
"Information geometric analysis of phase transitions in complex patterns: the case of the Gray-Scott reaction-diffusion model",
J. Stat. Mech. Theory Exp., 2016(04), 043301.
DOI: 10.1088/1742-5468/2016/04/043301